In [1]:
import os,numpy as np,re
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from tensorflow.keras import models
import tensorflow as tf
import cv2
from model import predict_img
tf.__version__

'2.3.0'

In [3]:
#creating train and test directories
try:
    os.mkdir("Image_data")
except FileExistsError:
    for dir_ in ["Image_data\\Train","Image_data\\Test","Image_data\\Complete_images"]:
        try:
            os.mkdir(dir_)
        except FileExistsError:
            print(f"Directory/file '{dir_}' already created")

Directory/file 'Image_data\Train' already created
Directory/file 'Image_data\Test' already created
Directory/file 'Image_data\Complete_images' already created


In [4]:
#creating the training,validation and test data sets
Train_1="Image_data\\Train\\2_digits"
Train_2="Image_data\\Train\\1_digit"
Test_1="Image_data\\Test\\2_digits"
Test_2="Image_data\\Test\\1_digit"
Valid_1="Image_data\\Valid\\2_digits"
Valid_2="Image_data\\Valid\\1_digit"
Train=[Train_1,Train_2]
Test=[Test_1,Test_2]
Valid=[Valid_1,Valid_2]
for path in [Train,Test,Valid]:
    for label_path in path:
        try:
            os.mkdir(label_path)
        except FileExistsError:
            print(f"The file/directory {label_path} already exists")
            pass

The file/directory Image_data\Train\2_digits already exists
The file/directory Image_data\Train\1_digit already exists
The file/directory Image_data\Test\2_digits already exists
The file/directory Image_data\Test\1_digit already exists
The file/directory Image_data\Valid\2_digits already exists
The file/directory Image_data\Valid\1_digit already exists


In [6]:
def rename_images()->None:
    model_paths=[".model_wts(10-30).hdf5",".model_wts(1-9).hdf5"]
    mode=[models.load_model(path_) for path_ in model_paths]
    downloaded_img_path="C:\\Users\\DELL\\Pictures\\Sidegig\\Sidegigcaptcha7"
    new_images=os.listdir(downloaded_img_path)
    for imge in new_images:
        if os.path.isdir(downloaded_img_path+f"\\{imge}"):
            continue
        height, width, channels=36,60,4
        img_arr=cv2.imread(downloaded_img_path+f"\\{imge}")
        img_arr=cv2.cvtColor(img_arr,cv2.COLOR_BGR2RGBA)
        img_full_resized=cv2.resize(img_arr,(700,500))
        try:
            img_arr=img_arr.reshape(height, width*2,channels)
        except ValueError:
            continue
        img_arr_1=img_arr[:,:img_arr.shape[1]//2,:]
        img_arr_2=img_arr[:,img_arr.shape[1]//2:,:]
        prop_names=[]
        for i,img_array in enumerate([img_arr_1,img_arr_2]):
            img_array=img_array.reshape(1,height, width,channels)
            img_array=tf.constant(img_array)
            classes=os.listdir(Train[i])
            cls_map={idx:int(j) for idx, j in enumerate(classes)}
            prop_name=predict_img(img_array/255,model=mode[i],mapp=cls_map)
            
            prop_names.append(int(prop_name))
        cv2.putText(img_full_resized,f"{prop_names[0]}_{prop_names[1]}",(150,100),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
        cv2.imshow("Image",img_full_resized)
        cv2.waitKey(300)
        corr=input("are the values displayed in red on the image the same as the numbers in the image?" 
            "\nif yes press enter,please make sure the number displayed is correct before hitting enter,\nelse write the correct digit in the form 'digit_digit' e.g 20_8, and hit enter \n")
        cv2.destroyAllWindows()
        if corr=="":
            if os.path.exists(f"{downloaded_img_path}\\train_\\")==False:
                
                os.mkdir(f"{downloaded_img_path}\\train_\\")
            os.rename(downloaded_img_path+f"\\{imge}",
            f"{downloaded_img_path}\\train_\\{prop_names[0]}_{prop_names[1]} {imge}")
        else:
            match=re.search("^\d{2}_\d$",corr)
            while not match:
                cv2.putText(img_full_resized,f"{prop_names[0]}_{prop_names[1]}",(150,100),cv2.FONT_HERSHEY_COMPLEX,2,(0,0,255),2)
                cv2.imshow("Image",img_full_resized)
                cv2.waitKey(300)
                corr=input("Please make sure the value being written is in the form 'digits_digit'" 
                "e.g: 20_4, 15_9\nalso ensure the value is the correct one")
                match=re.search("^\d{2}_\d$",corr)
                cv2.destroyAllWindows()
            else:
                if os.path.exists(f"{downloaded_img_path}\\test_\\")==False:
                    os.mkdir(f"{downloaded_img_path}\\test_\\")
                os.rename(downloaded_img_path+f"\\{imge}",
                f"{downloaded_img_path}\\test_\\{corr} {imge}")
    return
rename_images()
CAPTCHA_PATH=r"C:\Users\DELL\Pictures\Sidegig\Sidegigcaptcha7"
len_train=len(os.listdir(CAPTCHA_PATH+"\\"+"train_"))
len_test=len(os.listdir(CAPTCHA_PATH+"\\"+"test_"))
tot_len=len_test+len_train
print("test_accuracy = ",len_train/tot_len)


test_accuracy =  0.9071131290200529


In [7]:
#path to scraped captcha images,each image consists of 2 numbers,a 2-digit number and a 1-digit number e.g 20_1
CAPTCHA_PATH=r"C:\Users\DELL\Pictures\Sidegig\Sidegigcaptcha7"
WORK_IMAGE_DIR="Image_data\\Complete_images"
images_folders=os.listdir(CAPTCHA_PATH)
#move images from download/rename directory to cwd 
for dir in images_folders:
    if os.path.isdir(CAPTCHA_PATH+"\\"+dir):
        for images in os.listdir(CAPTCHA_PATH+"\\"+dir):
            os.rename(f"{CAPTCHA_PATH}\\{dir}\\{images}",
                    f"{WORK_IMAGE_DIR}\\{dir}\\{images}")

#the WORK_IMAGE_DIR now contains two directories\folders
#train_:consists of images that were correctly classified by the last trained model
#test_: images that were incorrectly classified

In [8]:

Train_tst_dir=os.listdir(WORK_IMAGE_DIR)
#removing redundant information(folder name/time_stamp) and leaving only the image labels
#Note: images have been labelled manually to include their labels in the image file_name
#Note: after manually labelling the images initially, model was trained *see model.py* and then new images which were scraped 
#were labelled using the trained model *see rename_images()*.
for dirs in Train_tst_dir:
    Images=os.listdir(WORK_IMAGE_DIR+"\\"+dirs)
    Images=[image for image in Images if not os.path.isdir(WORK_IMAGE_DIR+"\\"+dirs+"\\"+image)]
    
    labels=[re.sub("[A-Za-z\(].+","",image_name).strip() for image_name in Images]
    labels=[re.search("(\d+)_(\d)",label) for label in labels]
    labels=[label.groups() for label in labels]
    labels
    for i, image in enumerate(Images):
        #split and separate the two-number images into separate folders containing single numbers

        img=plt.imread(WORK_IMAGE_DIR+"\\"+dirs+"\\"+image)
        img_2=img[:,img.shape[1]//2:,:]
        img_1=img[:,:img.shape[1]//2,:]
        #plt.imshow(img)
        try:
            os.mkdir(WORK_IMAGE_DIR+"\\"+dirs+"\\2_digits")
            os.mkdir(WORK_IMAGE_DIR+"\\"+dirs+"\\1_digit")
            os.mkdir(WORK_IMAGE_DIR+"\\"+dirs+"\\Full_image")
        except FileExistsError:
            pass
        plt.imsave(f"{WORK_IMAGE_DIR}\\{dirs}\\2_digits\\{labels[i][0]} ({i}).png",img_1)
        plt.imsave(f"{WORK_IMAGE_DIR}\\{dirs}\\1_digit\\{labels[i][1]} ({i}).png",img_2)
        os.rename(WORK_IMAGE_DIR+"\\"+dirs+"\\"+image,WORK_IMAGE_DIR+"\\"+dirs+"\\Full_image\\"+image)

In [9]:

for fold in os.listdir(WORK_IMAGE_DIR):
    labe_=os.listdir(f"{WORK_IMAGE_DIR}\\{fold}\\2_digits")
    labe_=[f_name for f_name in labe_ if not os.path.isdir(f"{WORK_IMAGE_DIR}\\{fold}\\2_digits\\{f_name}")]
    labe_2=os.listdir(f"{WORK_IMAGE_DIR}\\{fold}\\1_digit")
    labe_2=[f_name for f_name in labe_2 if not os.path.isdir(f"{WORK_IMAGE_DIR}\\{fold}\\1_digit\\{f_name}")]
    for file_name in labe_:
        new_label=re.search("^\d+",file_name).group()
        #create sub folders for each unique label 
        if os.path.exists(f"{WORK_IMAGE_DIR}\\{fold}\\2_digits\\{new_label}")==False:
            os.mkdir(f"{WORK_IMAGE_DIR}\\{fold}\\2_digits\\{new_label}")
        #move images into their corresponding label subfolder
        os.rename(f"{WORK_IMAGE_DIR}\\{fold}\\2_digits\\{file_name}",
        f"{WORK_IMAGE_DIR}\\{fold}\\2_digits\\{new_label}\\{file_name[:-4]}{time()}.png")
    for file_name in labe_2:
        new_label_2=re.search("^\d",file_name).group()
        if os.path.exists(f"{WORK_IMAGE_DIR}\\{fold}\\1_digit\\{new_label_2}")==False:
            os.mkdir(f"{WORK_IMAGE_DIR}\\{fold}\\1_digit\\{new_label_2}")
        os.rename(f"{WORK_IMAGE_DIR}\\{fold}\\1_digit\\{file_name}",
        f"{WORK_IMAGE_DIR}\\{fold}\\1_digit\\{new_label_2}\\{file_name[:-4]}{time()}.png")

In [10]:
from collections import defaultdict
min_num_images=defaultdict(list)
for d_set in os.listdir(WORK_IMAGE_DIR):

    two_digits=f"{WORK_IMAGE_DIR}\\{d_set}\\2_digits"
    one_digit=f"{WORK_IMAGE_DIR}\\{d_set}\\1_digit"
    
    for digit_type in (two_digits,one_digit):
        labels_=os.listdir(digit_type)
        
        for i,cls_label in enumerate(labels_):
            dir_images=os.listdir(digit_type+"\\"+cls_label)
            image_length=len(dir_images)
            if i==0:
                min_length=image_length
            if image_length< min_length:
                min_length=image_length

        min_num_images[d_set].append(min_length)
print(min_num_images)

defaultdict(<class 'list'>, {'test_': [13, 45], 'train_': [201, 499]})


In [11]:
from time import time
#percentage cut-off of images to be moved to validation and test sets
#note the higher percentage cut-offs in dict["test_"] as compared to dict["train"] 
#this is to test whether the subsequent models are performing better on the test set as model retraining continues
valid_perc={"test_":0.35,"train_":0.1}
test_perc={"test_":0.2,"train_":0.05}


for d_set in os.listdir(WORK_IMAGE_DIR):
    two_digits=f"{WORK_IMAGE_DIR}\\{d_set}\\2_digits"
    one_digit=f"{WORK_IMAGE_DIR}\\{d_set}\\1_digit"
    for dig_typ_idx,digit_type in enumerate((two_digits,one_digit)):
        for cls_label in os.listdir(digit_type):
            
            if os.path.exists(f"{Train[dig_typ_idx]}\\{cls_label}")==False:
                os.mkdir(f"{Train[dig_typ_idx]}\\{cls_label}")

            if os.path.exists(f"{Test[dig_typ_idx]}\\{cls_label}")==False:
                os.mkdir(f"{Test[dig_typ_idx]}\\{cls_label}")

            if os.path.exists(f"{Valid[dig_typ_idx]}\\{cls_label}")==False:
                os.mkdir(f"{Valid[dig_typ_idx]}\\{cls_label}")

            for image_num,image in enumerate(os.listdir(digit_type+"\\"+cls_label)):
                #move images into training set based on the cut-offs
                if image_num<= min_num_images[d_set][dig_typ_idx]*(1-valid_perc[d_set]):
                    os.rename(f"{digit_type}\\{cls_label}\\{image}",
                             f"{Train[dig_typ_idx]}\\{cls_label}\\{image[:-4]}{time()}.png")
                #move images into Validation set based on the cut-offs
                elif image_num<=min_num_images[d_set][dig_typ_idx]*(1-test_perc[d_set]):
                    os.rename(f"{digit_type}\\{cls_label}\\{image}",
                             f"{Valid[dig_typ_idx]}\\{cls_label}\\{image[:-4]}{time()}.png")
                #move remaining images into test set 
                else:
                    os.rename(f"{digit_type}\\{cls_label}\\{image}",
                             f"{Test[dig_typ_idx]}\\{cls_label}\\{image[:-4]}{time()}.png")

In [12]:
#for Train and Validation sets,ensure the number of images in each class label sub_folder are equal 
len_images=defaultdict(set)
for d_set in[Train,Valid]:
    for idx,digit_type in enumerate(d_set):
        dataset=re.search(r"\\(.*$)",os.path.dirname(digit_type)).group(1)
        dig_typ_name=os.path.basename(digit_type)
        cls_labels=os.listdir(digit_type)
        for cls_label in cls_labels:
            dir_images=os.listdir(digit_type+"\\"+cls_label)
            image_length=len(dir_images)        
            len_images[dataset+"_"+dig_typ_name].add(image_length)
        if len(len_images[dataset+"_"+dig_typ_name])>1:
            min_length=min(len_images[dataset+"_"+dig_typ_name])
            for cls_label in cls_labels:
                dir_images=os.listdir(digit_type+"\\"+cls_label)
                image_length=len(dir_images)
                if image_length>min_length:
                    excess=image_length-min_length
                    [os.rename(digit_type+"\\"+cls_label+"\\"+dir_images[i],
                    Test[idx]+"\\"+cls_label+"\\"+dir_images[i]) for i in range(excess)]

print(len_images)

defaultdict(<class 'set'>, {'Train_2_digits': {402}, 'Train_1_digit': {1011}, 'Valid_2_digits': {46}, 'Valid_1_digit': {123}})
